# Wordle Solver

In [1]:
from utils import *
%load_ext autoreload
%autoreload 2

In [2]:
# Initialize Wordle class
wordle = Wordle()

# Main function that returns a list of possible words
green_letters = 'ILBU'
green_positions = [3,4,1,2]
yellow_letters = 'D'
yellow_positions = [4,]
bad_letters = 'CRANESOYNT'

# Find words in a list of possible words
words = wordle.find_words(green_letters, green_positions, yellow_letters, yellow_positions, bad_letters)
print(words)

['build']


In [3]:
# Repetitive letters
letters_df = wordle.repetitive_letters(wordle_list=words)
print(letters_df.to_string(index=False))

Letters  Count
      B      1
      U      1
      I      1
      L      1
      D      1


#### Choosing next word

In [4]:
# Main function that returns a list of possible words
green_letters = ''
green_positions = []
yellow_letters = 'UTBD'
yellow_positions = []
bad_letters = ''

# Find words in a list of possible words
words = wordle.find_words(green_letters, green_positions, yellow_letters, yellow_positions, bad_letters, answer_word_list=False)
print(words)

['bundt', 'debut', 'doubt', 'tubed']


In [5]:
# help(score)

In [6]:
df = wordle.load_data(file_name='wordle_ranking.csv')
df

,Names,Games_Won
0,Murilo,230
1,Barbara,93
2,Draw,233


In [7]:
# Update score
score_df = wordle.score()
score_df

,Names,Games_Won
0,Murilo,230
1,Barbara,93
2,Draw,234


In [8]:
# Print results
print(score_df.to_string(index=False))

  Names  Games_Won
 Murilo        230
Barbara         93
   Draw        234


In [9]:
# help(wordle.reset_score)

#### Set custom score

In [10]:
# help(wordle.set_score)

In [11]:
# # Set custom score
# df = wordle.set_score(m_score=196, b_score=79, draw_score=204)
# df

#### Reset score

In [12]:
## Reset score
# wordle.reset_score()